In [2]:
import nltk
import numpy as np
import pandas as pd
import underthesea
import re
from collections import Counter

# Utils

In [ ]:
def read_file(file_name):
    return open(file_name, 'r', encoding='utf-8').read().split('\n')

def write_file(file_name, text):
    file = open(file_name, 'w', encoding='utf-8')
    for sentences in text:
        file.writelines(sentences + '\n')

In [3]:
def inspect_tag(tag) -> bool:
    pattern = re.compile(r'^[A-Z][a-z]?$')
    return bool(pattern.search(tag))

In [4]:
def inspect_word(word) -> bool:
    pattern = re.compile(r'[!"#$%&\'()*+,-./:;<=>?@\[\]^`{|}~“”\\]+')
    return not(bool(pattern.search(word)))

In [ ]:
def get_raw_text(filename):
    """
    - trả về data nguyên bản (kể cả các invalid word, tag) trong file
    - bên cạnh đó, xử lí word và tag -> tách tag và word sau đó kẹp thành 1 tuple như này (word, tag)
    - loại bỏ các word không tag hoặc word có nhiều hơn 1 tag
    """ 
    text = read_file(filename)
    data = []

    for sentence in text:
        for duo in sentence.strip().split(' '):
            data.append(tuple(duo.split('/')))

    data = [duo for duo in data if len(duo) == 2]
    return data

In [ ]:
def get_processed_data(filename):
    opening = ('<s>', '<s>')
    ending = ('</s>', '</s>')

    corpus = get_raw_text(filename)
    refreshed_data = []
    sentence = []

    for word, tag in corpus:
        if inspect_word(word) and inspect_tag(tag):
            sentence.append((word, tag))
        elif word in ['.', '...', ':', '!', '?', '-', '"']:
            
            refreshed_data.append([opening] + sentence + [ending])
            sentence.clear()

    return refreshed_data

In [ ]:
def load_corpus(filename):
    data = get_processed_data(filename)
    corpus = []
    for sentence in data:
        corpus.extend(sentence)
    return corpus

# Viterbi Algorithm